In [ ]:
##### This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

In [ ]:
tokens = tokenizer.encode('It was good but couldve been better. Great', return_tensors='pt')

In [ ]:
result = model(tokens)

In [ ]:
result.logits

In [ ]:
int(torch.argmax(result.logits))+1

In [ ]:
import pandas as pd

# Load your CSV file containing movie reviews
file_path = '/kaggle/input/clapper-massive-rotten-tomatoes-movies-and-reviews/rotten_tomatoes_movie_reviews.csv'  # Update this with your file path
data = pd.read_csv(file_path)

# Assuming your CSV file has a column named 'reviewText' containing movie reviews
reviews_column = 'reviewText'

# Extracting the reviews and creating a DataFrame
reviews_df = pd.DataFrame(data[reviews_column])

# Display the DataFrame containing movie reviews
print(reviews_df)

In [ ]:
reviews_df['reviewText'].iloc[1]

In [ ]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [ ]:
sentiment_score(reviews_df['reviewText'].iloc[1])

In [ ]:
import pandas as pd

# Load your reviews DataFrame
# Assuming you already have 'reviews_df' DataFrame with movie reviews
# reviews_df = pd.read_csv('path/to/your/reviews_dataframe.csv')

# Load the CSV file containing the 'movies' column
movies_file_path = '/kaggle/input/clapper-massive-rotten-tomatoes-movies-and-reviews/rotten_tomatoes_movie_reviews.csv'  # Update with your file path
movies_data = pd.read_csv(movies_file_path)

# Assuming the CSV file contains a column named 'movies' with movie names

# Extract the 'movies' column from the CSV file
movies_column = movies_data['id']

# Insert the 'movies' column as the first column in the 'reviews_df' DataFrame
reviews_df.insert(0, 'movies', movies_column)

# Display the updated DataFrame with 'movies' as the first column
print(reviews_df)

In [ ]:
reviews_df = reviews_df.head(50)

In [ ]:
import pandas as pd

# Load your reviews DataFrame containing the 'movies' column
# Assuming you already have 'reviews_df' DataFrame with 'movies' column
# reviews_df = pd.read_csv('path/to/your/reviews_dataframe.csv')

# Load the first CSV file containing 'id' column
file1_path = '/kaggle/input/clapper-massive-rotten-tomatoes-movies-and-reviews/rotten_tomatoes_movie_reviews.csv'
file1_data = pd.read_csv(file1_path)

# Load the second CSV file containing 'id' and 'title' columns
file2_path = '/kaggle/input/clapper-massive-rotten-tomatoes-movies-and-reviews/rotten_tomatoes_movies.csv'
file2_data = pd.read_csv(file2_path)

# Assuming 'file1_data' DataFrame contains 'id' column
# Assuming 'file2_data' DataFrame contains 'id' and 'title' columns

# Iterate over 'movies' column in the reviews DataFrame to match IDs and add movie names
movies_list = []
for index, row in reviews_df.iterrows():
    current_movie_id = row['movies']
    
    # Search for the matching ID in file1_data
    matching_row_file1 = file1_data[file1_data['id'] == current_movie_id]
    
    if not matching_row_file1.empty:
        corresponding_id_file2 = matching_row_file1['id'].values[0]
        
        # Find the corresponding movie name in file2_data using the matched ID
        matching_row_file2 = file2_data[file2_data['id'] == corresponding_id_file2]
        
        if not matching_row_file2.empty:
            corresponding_movie_name = matching_row_file2['title'].values[0]
            movies_list.append(corresponding_movie_name)
        else:
            movies_list.append(None)  # If no corresponding movie name found, append None
    else:
        movies_list.append(None)  # If no matching ID found, append None

# Add the list of corresponding movie names as the first column to the reviews DataFrame
reviews_df.insert(0, 'Movie_Name', movies_list)

# Drop the 'id' column from the reviews DataFrame
reviews_df.drop('movies', axis=1, inplace=True)

# Display the updated DataFrame with the added movie names as the first column and 'id' column dropped
print(reviews_df)

In [ ]:
def apply_sentiment(x):
    if isinstance(x, str):  # Check if the value is a string
        return sentiment_score(x[:512])  # Perform sentiment analysis on the first 512 characters
    else:
        return None  # Return None for non-string values

In [ ]:
reviews_df['rating'] = reviews_df['reviewText'].apply(apply_sentiment)

In [ ]:
reviews_df